In [3]:
from moabb import paradigms as moabb_paradigms
from moabb.evaluations import (
    MixedSubjectSingleEvaluation,
    CrossSubjectSingleEvaluation,
)
from moabb.datasets import FER

# %load_ext autoreload
# %autoreload 2

data_path = '/Users/ldix/Documents/Projects/fer_BIDS'
evaluation = MixedSubjectSingleEvaluation
dataset = FER(data_path)
paradigm = moabb_paradigms.DiscreteEmotions()
pipefile = "/Users/ldix/Documents/Projects/moabb-generalised/pipelines/braindecode_shallowFSCBPNet.yml"

# suffix = f"ds-{dataset_name}_pipe-{run_configs['pipeline']}_fold-{run_configs['fold']}"


Loading BIDS layout...
BIDS layout loaded.


In [7]:
res = evaluation(
    paradigm=paradigm,
    datasets=dataset,
    # n_splits=configs['n_splits'],
    # random_state=configs['random_state'],
    # overwrite=configs['overwrite'],
    # save_model=configs['save_model'],
    # hdf5_path=configs['results_folder'],
    # suffix=suffix,
    # cache_config=configs['cache_config'],
    fold=0#run_configs['fold']
 ).process(
    pipefile=pipefile
)

print(f"Results: {res}")

TypeError: SingleEvaluation.__init__() missing 1 required positional argument: 'fold'

In [3]:
from pathlib import Path
from bids import BIDSLayout

data_path = Path('/Users/ldix/Documents/Projects/fer_BIDS')
l = BIDSLayout(data_path)

In [5]:
l.get_sessions()

[]

In [ ]:
event_files = l.get(suffix='eeg', run=[], session=[], extension=['.edf'], subject='cxy')
event_files[0].get_entities()
# valid_subjects = list(set([v.entities['subject'] for v in eggs]))
# valid_subjects

{'datatype': 'eeg',
 'extension': '.edf',
 'run': 0,
 'subject': 'cxy',
 'suffix': 'eeg',
 'task': 'fer'}

In [1]:
from moabb.datasets import BIDS
data_path = '/Users/ldix/Documents/Projects/fer_BIDS'
dataset = BIDS(BIDS_path=data_path)

Loading BIDS layout...
BIDS layout loaded.


In [ ]:
import mne
egg = dataset.layout.get(subject='cxy', suffix='eeg', extension='edf', run='1')[0]
raw = mne.io.read_raw_edf(egg, preload=True, verbose=False)


In [ ]:
import pandas as pd
import mne

datafile = egg.path
channels_file = dataset.layout.get_nearest(path = egg.path.replace('eeg.edf','channels.tsv'), suffix='channels', return_type='id')

channels = pd.read_csv(channels_file.path, sep='\t')
info = mne.create_info(ch_names=channels['name'].values.tolist(), sfreq=raw.info['sfreq'], ch_types=channels['type'].values.tolist())
raw = mne.io.read_raw_edf(datafile, preload=True, verbose=False, info=info)

KeyError: "kind must be one of ['grad', 'mag', 'ref_meg', 'eeg', 'seeg', 'dbs', 'ecog', 'eog', 'emg', 'ecg', 'resp', 'bio', 'misc', 'stim', 'exci', 'syst', 'ias', 'gof', 'dipole', 'chpi', 'fnirs_cw_amplitude', 'fnirs_fd_ac_amplitude', 'fnirs_fd_phase', 'fnirs_od', 'hbo', 'hbr', 'csd', 'temperature', 'gsr', 'eyegaze', 'pupil'], not EEG"

In [31]:
criteria = egg.get_entities()
criteria
# criteria['']
# dataset.layout.get()

{'datatype': 'eeg',
 'extension': '.edf',
 'run': 1,
 'subject': 'cxy',
 'suffix': 'eeg',
 'task': 'fer'}

In [26]:
raw.info['temp'] = egg.get_metadata()
raw.info['temp']
# egg.get_metadata()

{'CapManufacturer': 'EasyCap',
 'CapManufacturersModelName': 'BrainCap',
 'ECGChannelCount': 0,
 'EEGChannelCount': 63,
 'EEGGround': 'neck',
 'EEGPlacementScheme': '10-20',
 'EEGReference': 'left earlobe',
 'EMGChannelCount': 0,
 'EOGChannelCount': 0,
 'HardwareFilters': 'n/a',
 'Manufacturer': 'g.tec',
 'ManufacturersModelName': 'g.HIamp-Research',
 'MiscChannelCount': 0,
 'PowerLineFrequency': 50,
 'RecordingDuration': 1365.0,
 'RecordingType': 'continuous',
 'SamplingFrequency': 256.0,
 'SoftwareFilters': {'Highpass': {'Cut-off frequency': 0.5, 'Filter order': 4},
  'Lowpass': {'Cut-off frequency': 100, 'Filter order': 4},
  'Notch': {'Cut-off frequency': 50, 'Filter order': 4}},
 'SubjectArtefactDescription': 'n/a',
 'TaskName': 'FER',
 'TriggerChannelCount': 0}

In [ ]:
### GETTING EVENTS


sfreq = raw.info["sfreq"]
BIDS_file = self.layout.get_file(raw.filenames[0])
subj = BIDS_file.entities["subject"]
run = BIDS_file.entities["run"]
events_file = self.layout.get(suffix='events', subject=subj, run=run)[0].path
events_df = pd.read_csv(events_file, sep="\t")

events_df = events_df[events_df['flag'] == "video"]
onset = events_df['onset'].values * sfreq
description = events_df['trial_type'].apply(lambda x: self.event_id[x]).values

events = np.vstack([onset, np.zeros_like(onset), description]).T
events = events.astype(int)
